In [1]:
import numpy as np
import os
from PIL import Image
import pickle
from keras.preprocessing import image
from keras.models import load_model

Using TensorFlow backend.


In [2]:
path ="val/data"
label = "val/val_label.txt"


height = 45
width = 45
dirlist = []
for root, dirs, files in os.walk(path):
    for dir in dirs :
        dirlist.append(dir)
dirlist.sort()
print(len(dirlist ))

with open(label) as file_object:
    p = file_object.readlines()



300


In [3]:
num = 0
X1 = np.zeros((len(dirlist)*36,45,45,3), dtype = np.uint8)
X2 = np.zeros((len(dirlist)*36,45,45,3), dtype = np.uint8)
y = np.zeros((len(dirlist)*36, 1), dtype = np.uint8)

for dir in dirlist:
    images1 = []
    
    
    image1 = Image.open(path+'/'+dir+'/'+dir+'.jpg')
    image1 = image1.resize((180,45))

                    
    for i in range(4):
        left = 45*i
        right = 45 * (i+1)
        box = (left,0,right,45)
        data = np.asarray(image1.crop(box))
        images1.append(data)
        
    
                    
    for l in p:
        if l[:4] == dir:
            break;

    for i in range(4):
        for j in range(9):
            im = Image.open(path+'/'+dir+'/'+str(j)+'.jpg')
            im = np.asarray(im)
            X1[num] = images1[i]
            X2[num] = im
            if int(l[5+i]) == j:
                y[num] = 1
            num += 1


In [4]:
model = load_model('fifth_model.h5')    
def testCaptcha(index):
    true_label = p[index][5:9]
    X_test1 = np.zeros((9, 45, 45, 3), dtype = np.float32)
    X_test2 = np.zeros((9, 45, 45, 3), dtype = np.float32)
    s = ''
    for i in range(4):
        for j in range(9):
            X_test1[j] = X1[index*36+i*9+j]
            X_test2[j] = X2[index*36+i*9+j]
        result = model.predict([X_test1,X_test2])
        a = str(np.argmax(result))
        s += a
            
        
    if s==true_label:
        return 1
    else:
        return 0
    
c=0
for i in range(300):
     if testCaptcha(i)==1:
                c+=1
print(c/300)



0.96
